<a href="https://colab.research.google.com/github/aggarwalakshay2509/ml/blob/main/SarcasticSentimentRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

In [ ]:
with open('sarcasm.json','r') as f:
  dataset=json.load(f)
print(type(dataset))
sentences=[]
labels=[]
urls=[]
for item in dataset:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])
  urls.append(item['article_link'])

<class 'list'>


In [ ]:
print(sentences[0])
print(labels[0])

former versace store clerk sues over secret 'black code' for minority shoppers
0


In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
training_size=20000
training_sentences=sentences[:training_size]
testing_sentences=sentences[training_size:]
training_labels=labels[:training_size]
testing_labels=labels[training_size:]
training_labels=np.array(training_labels)
testing_labels=np.array(testing_labels)

In [ ]:
vocab_size=20000
max_length=30

tokenizer=Tokenizer(num_words=vocab_size,oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
word_index=tokenizer.word_index

training_sequences=tokenizer.texts_to_sequences(training_sentences)
training_padded=pad_sequences(training_sequences,padding="post",maxlen=max_length)

testing_sequences=tokenizer.texts_to_sequences(testing_sentences)
testing_padded=pad_sequences(testing_sequences,padding="post",maxlen=max_length)

print(vocab_size,max_length)

print(training_padded[0])

print(training_padded.shape)

20000 30
[  328 12776   799  3405  2404    47   389  2214 12777     6  2614  8863
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]
(20000, 30)


In [ ]:
def get_key(val):
  for key,value in word_index.items():
    if value==val:
      return key

In [ ]:
print(training_sentences[0])
print(training_sequences[0])
print(training_padded[0])
print(word_index['former'])
print(type(training_padded[0]))
print(type(training_labels[0]))

former versace store clerk sues over secret 'black code' for minority shoppers
[328, 12776, 799, 3405, 2404, 47, 389, 2214, 12777, 6, 2614, 8863]
[  328 12776   799  3405  2404    47   389  2214 12777     6  2614  8863
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]
328
<class 'numpy.ndarray'>
<class 'numpy.int64'>


In [ ]:
model=keras.Sequential([
  keras.layers.Embedding(input_dim=vocab_size,output_dim=2,input_length=max_length),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(30,activation="relu"),
  keras.layers.Dense(1,activation="sigmoid")
])

model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

history=model.fit(training_padded,training_labels,epochs=5,verbose=2)

Epoch 1/5
625/625 - 1s - loss: 0.6678 - accuracy: 0.5816 - 1s/epoch - 2ms/step
Epoch 2/5
625/625 - 1s - loss: 0.4629 - accuracy: 0.8266 - 919ms/epoch - 1ms/step
Epoch 3/5
625/625 - 1s - loss: 0.3151 - accuracy: 0.8798 - 1s/epoch - 2ms/step
Epoch 4/5
625/625 - 1s - loss: 0.2497 - accuracy: 0.9064 - 925ms/epoch - 1ms/step
Epoch 5/5
625/625 - 1s - loss: 0.2049 - accuracy: 0.9230 - 933ms/epoch - 1ms/step


In [ ]:
model.evaluate(testing_padded,testing_labels)

210/210 [==============================] - 0s 1ms/step - loss: 0.3348 - accuracy: 0.8585


[0.33477261662483215, 0.8585482239723206]

In [ ]:
sentence=['Mom Warns Son to ‘Watch Out for Idiots,’ Rear‑Ends His Motorcycle.',
'CIA director William Burns says ties between Russia, Iran suffer from "limits," lack of trust'
]

sequence=tokenizer.texts_to_sequences(sentence)
sequence_padded=pad_sequences(sequence,maxlen=max_length,padding="post")

print(sequence_padded)
print(get_key(1))

[[ 153  551  304    2    1   24    6 7744    1    1   33 3762    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [1282  550 3904 2639   51 2184  379  466  549 4497   15 3549 1843    3
  2654    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]
<OOV>


In [ ]:
predicted_values=model.predict(sequence_padded)
for item in predicted_values:
  if item>0.5:
    print(item)
    print("Sarcastic")
  else:
    print(item)
    print("Not Sarcastic")

[0.84388745]
Sarcastic
[0.05980021]
Not Sarcastic
